In [9]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score,confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn import model_selection

from warnings import filterwarnings
filterwarnings('ignore')

In [10]:
df = pd.read_csv(r"C:\Users\LENOVO\Desktop\KodluyoruzBootcamp\Ödevler Ve Çalışmalar\hmelq.csv")

In [11]:
import Classes1 as cl

In [12]:

df.head()

,bad,loan,mortdue,value,reason,job,yoj,derog,delinq,clage,ninq,clno,debtinc
0,0,81200,18834.0,108355.0,HomeImp,NaN,28.0,0.0,0.0,139.14,0.0,14.0,34.042
1,0,12600,103960.0,127384.0,DebtCon,NaN,2.0,0.0,0.0,129.02,0.0,25.0,34.479
2,0,18000,46865.0,61266.0,DebtCon,NaN,5.0,0.0,0.0,102.59,2.0,9.0,26.354
3,0,10300,57676.0,71027.0,DebtCon,NaN,19.0,0.0,0.0,157.52,1.0,11.0,33.992
4,0,9400,56508.0,78358.0,DebtCon,NaN,17.0,0.0,0.0,141.93,0.0,11.0,32.327


In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bad,3576.0,0.199385,0.399594,0.00,0.00,0.000,0.00,1.000
loan,3576.0,18529.697987,11262.834060,1100.00,11000.00,16200.000,23100.00,89900.000
mortdue,3262.0,74013.907992,44016.833772,2619.00,46700.25,65858.000,91717.00,399550.000
value,3512.0,101076.922759,54865.275351,8800.00,65753.00,89183.000,118928.25,850000.000
yoj,3264.0,8.832644,7.568046,0.00,3.00,7.000,13.00,41.000
derog,3149.0,0.261670,0.865178,0.00,0.00,0.000,0.00,10.000
delinq,3225.0,0.432558,1.099230,0.00,0.00,0.000,0.00,15.000
clage,3397.0,179.832529,87.066198,0.49,115.60,173.640,230.71,1168.230
ninq,3273.0,1.186373,1.728723,0.00,0.00,1.000,2.00,17.000
clno,3443.0,21.507116,10.167490,0.00,15.00,20.000,26.00,71.000


In [14]:
df.isnull().sum()

bad          0
loan         0
mortdue    314
value       64
reason     147
job        167
yoj        312
derog      427
delinq     351
clage      179
ninq       303
clno       133
debtinc    767
dtype: int64

In [15]:
df.dropna(inplace=True)

In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bad,2018.0,0.088206,0.283665,0.00,0.0000,0.000,0.0000,1.000
loan,2018.0,19010.257681,10768.607345,1700.00,12000.0000,16900.000,23600.0000,89900.000
mortdue,2018.0,76337.035183,44476.368189,5076.00,49577.2500,67595.000,92922.7500,399412.000
value,2018.0,106899.244797,54026.707096,23031.00,71287.2500,94514.000,121220.7500,512650.000
yoj,2018.0,9.003964,7.573012,0.00,3.0000,7.000,13.0000,41.000
derog,2018.0,0.150149,0.580421,0.00,0.0000,0.000,0.0000,10.000
delinq,2018.0,0.264123,0.789816,0.00,0.0000,0.000,0.0000,10.000
clage,2018.0,180.645020,84.420116,0.49,118.2050,176.670,229.6925,1168.230
ninq,2018.0,1.037661,1.578414,0.00,0.0000,0.000,2.0000,13.000
clno,2018.0,22.265114,9.457238,0.00,16.0000,21.000,27.0000,64.000


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2018 entries, 153 to 3549
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   bad      2018 non-null   int64  
 1   loan     2018 non-null   int64  
 2   mortdue  2018 non-null   float64
 3   value    2018 non-null   float64
 4   reason   2018 non-null   object 
 5   job      2018 non-null   object 
 6   yoj      2018 non-null   float64
 7   derog    2018 non-null   float64
 8   delinq   2018 non-null   float64
 9   clage    2018 non-null   float64
 10  ninq     2018 non-null   float64
 11  clno     2018 non-null   float64
 12  debtinc  2018 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 220.7+ KB


In [18]:
from sklearn.preprocessing import LabelEncoder

df.dropna(inplace=True)

lbe = LabelEncoder()
df["reason"] = lbe.fit_transform(df["reason"])
df = pd.get_dummies(df,columns= ["job"], prefix= ["job"])

In [19]:
df.head(2)

,bad,loan,mortdue,value,reason,yoj,derog,delinq,clage,ninq,clno,debtinc,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self
153,0,18200,94727.0,136877.0,0,15.0,0.0,0.0,168.96,2.0,26.0,36.056,1,0,0,0,0,0
154,0,21700,79240.0,96784.0,0,5.0,0.0,0.0,64.51,6.0,24.0,38.079,1,0,0,0,0,0


In [20]:
y = df["bad"]
X = df.drop(labels=["bad"],axis=1)


X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    random_state = 42,
                                                    test_size = 0.25)

In [ ]:
YSA = cl.Methods()

YSA.YsaClassifier(X_train,y_train,X_test,y_test)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   54.2s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  9.3min finished


Tune Edilmemis Tahmin sonuclari Accuracy degeri:  0.9465346534653465
-------------------------------
Tune Edilmemis Confusion matrix sonuc: 
 [[457   9]
 [ 18  21]]
****************************************************
Tune sonrasi Tahmin sonuclari RMSE degeri: 0.9663366336633663
-------------------------------
Tune sonrasi Confusion matrix sonuc: 
 [[460   6]
 [ 11  28]]


In [22]:
df["ninq"].value_counts()

0.0     1014
1.0      480
2.0      278
3.0      140
4.0       40
5.0       15
6.0       14
7.0       11
10.0      10
8.0        8
9.0        6
13.0       1
11.0       1
Name: ninq, dtype: int64

In [23]:
a = df["ninq"] == 0
df["ninq"][a] = "a"

a = df["ninq"] == 1
df["ninq"][a] = "b"
a = df["ninq"] == 2
df["ninq"][a] = "b"
a = df["ninq"] == 3
df["ninq"][a] = "b"

a = df["ninq"] == 4
df["ninq"][a] = "c"
a = df["ninq"] == 5
df["ninq"][a] = "c"
a = df["ninq"] == 6
df["ninq"][a] = "c"

a = df["ninq"] == 7
df["ninq"][a] = "d"
a = df["ninq"] == 8
df["ninq"][a] = "d"
a = df["ninq"] == 9
df["ninq"][a] = "d"

a = df["ninq"] == 10
df["ninq"][a] = "e"
a = df["ninq"] == 11
df["ninq"][a] = "e"
a = df["ninq"] == 12
df["ninq"][a] = "e"
a = df["ninq"] == 13
df["ninq"][a] = "e"

In [24]:
df["ninq"].value_counts()

a    1014
b     898
c      69
d      25
e      12
Name: ninq, dtype: int64

In [25]:
df = pd.get_dummies(df,columns= ["ninq"], prefix= ["ninq"])

In [26]:
df.head(2)

,bad,loan,mortdue,value,reason,yoj,derog,delinq,clage,clno,...,job_Office,job_Other,job_ProfEx,job_Sales,job_Self,ninq_a,ninq_b,ninq_c,ninq_d,ninq_e
153,0,18200,94727.0,136877.0,0,15.0,0.0,0.0,168.96,26.0,...,0,0,0,0,0,0,1,0,0,0
154,0,21700,79240.0,96784.0,0,5.0,0.0,0.0,64.51,24.0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
y = df["bad"]
X = df.drop(labels=["bad"],axis=1)


X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    random_state = 42,
                                                    test_size = 0.25)


YSA = cl.Methods()

YSA.YsaClassifier(X_train,y_train,X_test,y_test)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   54.9s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  9.6min finished


Tune Edilmemis Tahmin sonuclari Accuracy degeri:  0.9405940594059405
-------------------------------
Tune Edilmemis Confusion matrix sonuc: 
 [[457   9]
 [ 21  18]]
****************************************************
Tune sonrasi Tahmin sonuclari RMSE degeri: 0.9524752475247524
-------------------------------
Tune sonrasi Confusion matrix sonuc: 
 [[457   9]
 [ 15  24]]


In [27]:
df_y = pd.read_excel(r"C:\Users\LENOVO\Desktop\KodluyoruzBootcamp\Ödevler Ve Çalışmalar\HW_Data_Set.xlsx")
df_y.head(2)

,ind_5,ind_6,ind_8,ind_9,ind_10,ind_12,ind_13,ind_14,ind_15,ind_16,...,ind_416,ind_418,ind_420,ind_422,ind_424,ind_426,ind_428,20_target,50_target,90_target
0,19,17,100.0,85.714286,14.285714,72.363515,60.808814,23.80,17.62,11.73,...,-49.6,-54,-152,-353,1.0,0.498547,0.701906,15.135802,35.625252,36.997753
1,24,19,100.0,78.571429,21.428571,74.275883,64.366798,11.45,18.16,12.22,...,-55.6,-60,-158,-359,1.0,0.537088,0.690833,15.143348,35.643013,37.016198


In [28]:
df_y.loc[df_y[df_y["ind_420"] == "?"].index] = np.nan 

# Degisken donusturmeden once icerisinde NAN degerler oldugu icin ilk once bu OBJECT tipindeki degiskenlerin NAN degerlerini degistiriyoruz
# Ardindan donusum gercelestiriyoruz

df_y["ind_422"] = df_y["ind_422"].fillna(df_y.mean()["ind_422"])
df_y["ind_420"] = df_y["ind_420"].fillna(df_y.mean()["ind_420"])

df_y["ind_422"] = df_y["ind_422"].astype('int32')
df_y["ind_420"] = df_y["ind_420"].astype('int32')

# Geriye kalan NaN degiskenleri de dolduruyoruz

column = list(df_y.select_dtypes(include=["int32","float64"]))
df_y[column] = df_y[column].apply(lambda x: x.fillna(x.mean()),axis=0)

In [29]:
df_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6167 entries, 0 to 6166
Columns: 136 entries, ind_5 to 90_target
dtypes: float64(133), int32(2), object(1)
memory usage: 6.4+ MB


In [30]:
df_y = pd.get_dummies(df_y,columns= ["ind_109"], prefix= ["ind_109"])

df_y = df_y.drop(["ind_109_RED"],axis=1)

df_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6167 entries, 0 to 6166
Columns: 136 entries, ind_5 to ind_109_GREEN
dtypes: float64(133), int32(2), uint8(1)
memory usage: 6.3 MB


In [ ]:
df_y.drop(labels=["20_target","90_target"],axis=1,inplace=True)


X = df_y.drop(labels="50_target",axis=1)
y = df_y["50_target"]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state = 42,shuffle = False)

YSA = cl.Methods()

YSA.YsaReg(X_train,y_train,X_test,y_test)

Fitting 10 folds for each of 96 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 22.5min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 63.3min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 84.3min finished


Tune Edilmemis Tahmin sonuclari RMSE degeri:  45.60909466505888
---------------------------------------------
Tune sonrasi Tahmin sonuclari RMSE degeri: 42.63284271311603
